In [ ]:
# very powerful, but discarded because requires at least 40gb cards to run and may be harder to tweak than diffusers
!git clone https://github.com/xdit-project/xDiT.git
%cd xDiT
%pip install flask
%pip install -e ".[diffusers,flash-attn]"
!python3 ./http-service/launch_host.py --config ./http-service/config.json
%cd /workspace/dtback

In [ ]:
# ComfyUI setup
%cd  /workspace/dtback
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd /workspace/dtback/ComfyUI
# !pip install -r requirements.txt
%cd /workspace/dtback/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager comfyui-manager
!git clone https://github.com/Suzie1/ComfyUI_Comfyroll_CustomNodes.git

!huggingface-cli download black-forest-labs/FLUX.1-Fill-dev flux1-fill-dev.safetensors --local-dir /workspace/dtback/ComfyUI/models/unet

!huggingface-cli download black-forest-labs/FLUX.1-dev ae.safetensors --local-dir /workspace/dtback/ComfyUI/models/vae
!huggingface-cli download comfyanonymous/flux_text_encoders clip_l.safetensors --local-dir /workspace/dtback/ComfyUI/models/clip
!huggingface-cli download comfyanonymous/flux_text_encoders t5xxl_fp16.safetensors --local-dir /workspace/dtback/ComfyUI/models/clip

!huggingface-cli download kudzueye/Boreal boreal-flux-dev-lora-v04_1000_steps.safetensors --local-dir /workspace/dtback/ComfyUI/models/loras

!curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
	| sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok
import base64
t = 'Mk9WSkJ6c2p4QzJzcVZLdEFJNjBWdndhWmhEXzd1UUMzVm8xZ3dMVmllS0JOcHdvNg=='
k = base64.b64decode(t.encode()).decode()
!ngrok config add-authtoken {k}
# ngrok http 8000

%cd /workspace/dtback
!python3 ComfyUI/main.py --port 8000

In [ ]:
# img2img amine
%cd ~
import time
start = time.time()
output = flux.img_to_img(
    prompt="""
    
    anime picture of two boys standing in a room near a table with pizza

    """,
    image="img2.jpg",
    strength=0.63,
    sample_steps=20,
    width=1024,
    height=768,
    seed=1000,
    cfg_scale=5,
    style_strength=20,
    sample_method="euler",
)
delta = round(time.time() - start)
output[0].convert('RGB').save("output_img.jpg")
from IPython.display import clear_output
clear_output()
print(f'took {delta} seconds')
output[0].show()

In [ ]:
#install decord
!sudo apt update
!sudo apt install libnvidia-decode-550-server -y
!sudo add-apt-repository ppa:jonathonf/ffmpeg-4 -y
!sudo apt-get update
!sudo apt-get install -y build-essential python3-dev python3-setuptools make cmake
!sudo apt-get install -y ffmpeg libavcodec-dev libavfilter-dev libavformat-dev libavutil-dev
!git clone --recursive https://github.com/dmlc/decord
%cd decord
!mkdir build
%cd build
!cmake .. -DUSE_CUDA=ON -DCMAKE_BUILD_TYPE=Release
!make
%cd ../python
!python3 setup.py install --user
%cd ../..

In [ ]:
import subprocess, zipfile, json
import pandas as pd

def download_tmdb_dataset():
    subprocess.run([
        "kaggle", "datasets", "download", "-d", "asaniczka/tmdb-movies-dataset-2023-930k-movies"
    ], check=True)
    zip_filename = "tmdb-movies-dataset-2023-930k-movies.zip"
    with zipfile.ZipFile(zip_filename, "r") as z:
        z.extractall(".")
    
    print("Dataset downloaded and extracted.")

download_tmdb_dataset()

df = pd.read_csv("TMDB_movie_dataset_v11.csv")
df_sorted = df.sort_values(by="vote_count", ascending=False)
titles = df_sorted.head(3000)['title'].to_numpy().tolist()

with open("./dataset_creation/movies.json", "w") as f:
    json.dump(titles, f, indent=4)

In [ ]:
# plot a cluster
import random, math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches

def display_cluster_images(cluster, face_metadata, images_per_row=5, max_images=60):
    if len(cluster) > max_images:
        cluster = random.sample(cluster, max_images)

    num_images = len(cluster)
    num_rows = math.ceil(num_images / images_per_row)
    fig, axes = plt.subplots(num_rows, images_per_row, figsize=(15, 3 * num_rows))
    axes = axes.flatten()

    for ax, idx in zip(axes, cluster):
        img_path = face_metadata[idx]["image_path"]
        img = mpimg.imread(img_path)
        ax.imshow(img)
        ax.axis("off")

        # Draw face bbox if available
        if "face_bbox" in face_metadata[idx] and face_metadata[idx]["face_bbox"]:
            fb = face_metadata[idx]["face_bbox"][0]
            x1, y1, x2, y2 = map(int, fb)
            rect_face = patches.Rectangle(
                (x1, y1), x2 - x1, y2 - y1,
                linewidth=2, edgecolor="red", facecolor="none"
            )
            ax.add_patch(rect_face)

        # Draw body bbox if available
        if "body_bbox" in face_metadata[idx] and face_metadata[idx]["body_bbox"]:
            bb = face_metadata[idx]["body_bbox"]
            x1, y1, x2, y2 = map(int, bb)
            rect_body = patches.Rectangle(
                (x1, y1), x2 - x1, y2 - y1,
                linewidth=2, edgecolor="blue", facecolor="none"
            )
            ax.add_patch(rect_body)

    for ax in axes[num_images:]:
        ax.axis("off")

    plt.tight_layout()
    plt.show()


display_cluster_images(clusters[6], metadata)